In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install rasterio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 70.3 MB/s eta 0:00:00


In [ ]:
import shutil
import os

# Set your folder name on Drive
drive_tif_path = "/content/drive/MyDrive/final_project/ONLY_TIF"
local_tif_path = "/content/ONLY_TIF"

# Copy recursively to Colab local runtime
if not os.path.exists(local_tif_path):
    shutil.copytree(drive_tif_path, local_tif_path)
else:
    print("✅ Local folder already exists")


In [ ]:
import pandas as pd

def make_triplet_csv(source_dir, output_csv_path):
    data = []
    id_counter = 1

    for root, _, files in os.walk(source_dir):
        files = [f for f in files if f.lower().endswith('.tif')]
        if not files:
            continue

        goes1 = goes2 = viirs = None
        for f in files:
            f_lower = f.lower()
            full_path = os.path.join(root, f)
            rel_path = os.path.relpath(full_path, source_dir)

            if 'geo16' in f_lower:
                goes1 = os.path.join(source_dir, rel_path)
            elif 'geo17' in f_lower:
                goes2 = os.path.join(source_dir, rel_path)
            elif 'geo18' in f_lower and goes2 is None:
                goes2 = os.path.join(source_dir, rel_path)
            elif 'combined' in f_lower:
                viirs = os.path.join(source_dir, rel_path)

        if goes1 and goes2 and viirs:
            data.append({
                'id': id_counter,
                'goes1_path': goes1,
                'goes2_path': goes2,
                'viirs_path': viirs,
            })
            id_counter += 1

    df = pd.DataFrame(data)
    df.to_csv(output_csv_path, index=False)
    print(f"✅ CSV saved to {output_csv_path} with {len(df)} triplets.")

make_triplet_csv("/content/ONLY_TIF", "/content/superres_triplets.csv")


✅ CSV saved to /content/superres_triplets.csv with 1260 triplets.


# first try

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import rasterio
import pandas as pd
from skimage.metrics import peak_signal_noise_ratio as psnr
from tqdm import tqdm
import json
import os

# Load fixed normalization ranges
with open("/content/radiance_visualization_ranges.json", "r") as f:
    fixed_ranges = json.load(f)

viirs_min = fixed_ranges["VIIRS"]["p0.5"]
viirs_max = fixed_ranges["VIIRS"]["p99.5"]
viirs_range = viirs_max - viirs_min

# Updated load_band with proper anomaly handling
def load_band(path):
    filename = os.path.basename(path).lower()
    is_viirs = "viirs" in filename or "combined_clip" in filename
    band_index = 1 if is_viirs else 7

    with rasterio.open(path) as src:
        image = src.read(band_index).astype(np.float32)

    mask = ~(np.isnan(image) | np.isinf(image))
    if np.any(mask):
        mean_val = image[mask].mean()
        image = np.where(mask, image, mean_val)
    else:
        image = np.zeros_like(image)

    return image

# Load triplets
df = pd.read_csv("/content/superres_triplets.csv")

baseline_psnrs = {}
psnr_values = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Computing normalized ESA baseline PSNR"):
    triplet_id = str(row["id"])

    goes1 = load_band(row["goes1_path"])
    goes2 = load_band(row["goes2_path"])
    viirs = load_band(row["viirs_path"])

    g1 = torch.from_numpy(goes1).unsqueeze(0).unsqueeze(0)
    g2 = torch.from_numpy(goes2).unsqueeze(0).unsqueeze(0)

    H, W = viirs.shape
    g1_hr = F.interpolate(g1, size=(H, W), mode='bicubic', align_corners=False)
    g2_hr = F.interpolate(g2, size=(H, W), mode='bicubic', align_corners=False)

    baseline_pred = ((g1_hr + g2_hr) / 2).squeeze().numpy()

    # Normalize using global VIIRS range
    viirs_np = (viirs - viirs_min) / viirs_range
    baseline_pred = (baseline_pred - viirs_min) / viirs_range

    viirs_np = np.clip(viirs_np, 0, 1)
    baseline_pred = np.clip(baseline_pred, 0, 1)

    score = psnr(viirs_np, baseline_pred, data_range=1.0)
    baseline_psnrs[triplet_id] = float(score)
    psnr_values.append(score)

mean_baseline = float(np.mean(psnr_values))

baseline_output = {
    "mean_baseline_psnr": mean_baseline,
    "per_triplet_scores": baseline_psnrs
}

with open("/content/esa_baseline_normalized_psnr.json", "w") as f:
    json.dump(baseline_output, f, indent=2)

print(f"✅ Normalized ESA baseline complete — mean PSNR: {mean_baseline:.8f} dB")
print("Saved to: /content/esa_baseline_normalized_psnr.json")


Computing normalized ESA baseline PSNR: 100%|██████████| 1260/1260 [01:14<00:00, 16.99it/s]

✅ Normalized ESA baseline complete — mean PSNR: 7.41 dB
Saved to: /content/esa_baseline_normalized_psnr.json


# second try

In [ ]:
"""
ESA‑style baseline cPSNR for one month (2023‑02) — *clean version*
------------------------------------------------------------------
Paste this as‑is.  It contains three additions compared with your
original script:

1.  Separate GOES and VIIRS normalisation ranges
2.  Kelvin ‘clear‑pixel’ quality mask + brightness‑bias cPSNR
3.  Shared cpsnr() helper (replaces compute_cpsnr)
"""

import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import rasterio
from tqdm import tqdm

# ---------------------------  RANGES  ----------------------------
with open("/content/radiance_visualization_ranges.json", "r") as f:
    _rng = json.load(f)

GOES_MIN = _rng["GOES"]["p2"]        # use p2 / p98 if you prefer
GOES_MAX = _rng["GOES"]["p98"]
GOES_RANGE = GOES_MAX - GOES_MIN

VIIRS_MIN = _rng["VIIRS"]["p2"]
VIIRS_MAX = _rng["VIIRS"]["p98"]
VIIRS_RANGE = VIIRS_MAX - VIIRS_MIN

# ------------------------  HELPER FUNCS  -------------------------
def load_band(path: str) -> np.ndarray:
    """Read one spectral band (GOES‑7 or VIIRS) as float32 numpy."""
    filename = os.path.basename(path).lower()
    is_viirs   = "viirs" in filename or "combined_clip" in filename
    band_index = 1 if is_viirs else 7                       # VIIRS‑I4 / GOES‑7
    with rasterio.open(path) as src:
        img = src.read(band_index).astype(np.float32)

    # replace NaN / ±inf with per‑image mean
    mask = ~(np.isnan(img) | np.isinf(img))
    img  = np.where(mask, img, img[mask].mean() if mask.any() else 0.0)
    return img

# ------------------------------------------------------------------

def cpsnr(gt: np.ndarray, pred: np.ndarray, mask: np.ndarray) -> float:
    """Corrected PSNR (Kelvin): brightness‑bias + clear‑pixel mask."""
    diff = (gt - pred) * mask
    b    = diff.sum() / (mask.sum() + 1e-8)                 # brightness bias
    cmse = ((gt - pred + b) ** 2 * mask).sum() / (mask.sum() + 1e-8)
    return -10.0 * np.log10(cmse + 1e-8)

# ---------------------------  MAIN LOOP  -------------------------
df  = pd.read_csv("/content/superres_triplets.csv")
df  = df[df["goes1_path"].str.contains("2023-02")].copy()   # Feb‑2023 only

triplet_scores, psnrs = {}, []

for _, row in tqdm(df.iterrows(),
                   total=len(df),
                   desc="Baseline cPSNR 2023‑02"):
    tid = str(row["id"])

    goes1 = load_band(row["goes1_path"])
    goes2 = load_band(row["goes2_path"])
    viirs = load_band(row["viirs_path"])

    # upscale GOES to VIIRS grid
    g1_hr = F.interpolate(torch.from_numpy(goes1)[None, None, ...],
                          size=viirs.shape, mode="bicubic",
                          align_corners=False).squeeze().numpy()
    g2_hr = F.interpolate(torch.from_numpy(goes2)[None, None, ...],
                          size=viirs.shape, mode="bicubic",
                          align_corners=False).squeeze().numpy()

    baseline_pred = 0.5 * (g1_hr + g2_hr)

    # --------------------  NORMALISE + CLIP  --------------------
    viirs_norm    = np.clip((viirs         - VIIRS_MIN) / VIIRS_RANGE,  0, 1)
    baseline_norm = np.clip((baseline_pred - GOES_MIN) / GOES_RANGE,   0, 1)

    # --------------------  CLEAR‑PIXEL MASK  --------------------
    cpsnr_val = cpsnr(viirs_norm, baseline_norm, np.ones_like(viirs_norm))

    triplet_scores[tid] = float(cpsnr_val)
    psnrs.append(cpsnr_val)

mean_cpsnr = float(np.mean(psnrs))

# ---------------------------  OUTPUT  ----------------------------
out = {
    "mean_baseline_cpsnr": mean_cpsnr,
    "per_triplet_scores":   triplet_scores
}
out_path = "/content/esa_baseline_2023_02_cpsnr.json"
with open(out_path, "w") as f:
    json.dump(out, f, indent=2)

print(f"✅ ESA baseline complete — mean cPSNR: {mean_cpsnr:.2f} dB")
print(f"   Saved to: {out_path}")


Baseline cPSNR 2023‑02: 100%|██████████| 29/29 [00:00<00:00, 37.32it/s]

✅ ESA baseline complete — mean cPSNR: 10.67 dB
   Saved to: /content/esa_baseline_2023_02_cpsnr.json
